In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math
from IPython.core.display import display,HTML
try:
    import cPickle as pickle     #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
#import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
    
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



# import plotly.plotly as py
# from plotly.graph_objs import *

# # i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
# #import plotly.tools as tls
# #tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


# import pygraphviz
# from networkx.drawing.nx_agraph import graphviz_layout




# ########## to be able to plot offline (without sending the plots to the plotly server every time)
# import plotly.offline as offline
# from plotly.graph_objs import *
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

# init_notebook_mode(connected=True)

In [ ]:
import plotly
plotly.__version__

In [2]:
path = "/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file = 'DROPBOX_only_activity_aggr_by_date_COMPLETE.csv'  # one line per user, folder, and date of activity

df = pd.read_csv(path+input_file, sep=',',na_values=["NAN","-1","null"],low_memory=False, parse_dates=['folder_creation_date','date']) # set header=0 if i wanna pass it my own list of header names
df.drop('Unnamed: 0', axis=1, inplace=True)

print df.shape



df=df.replace('nan', np.nan)  ## for some reason, there are diff nomenclatures for NANs, and it is not interpreting them correctly when first reading the file 
df=df.replace('NAN', np.nan)
df=df.replace('NaN', np.nan)
df=df.replace('-1', np.nan)
df=df.replace('', np.nan)




#convert the date columns from str to datetime    (for some reason, parsing when i read it does nothing)
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['folder_creation_date'] = pd.to_datetime(df['folder_creation_date'], errors='coerce')


df= df[df.date.notnull()]    
print "after removing rows without dates:\n  num. users:",len(df.user_id.unique()),df.shape    # 400000  (5141758, 7)











######## load dictionary aggregated info by user                                
pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_user_id_user_attr_new_COMPLETE.pickle'
with open(pickle_name, 'rb') as handle:
    dict_user_id_user_attr = pickle.load(handle)
print len(dict_user_id_user_attr.keys())

df_users = pd.DataFrame.from_dict(dict_user_id_user_attr,orient='index')
df_users['user_id'] = df_users.index
print df_users.shape   # 440353, 11







pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_list_users_COMPLETE.pickle'
with open(pickle_name, 'rb') as handle:
        dict_folder_list_users = pickle.load(handle)
print "read:", pickle_name, len(dict_folder_list_users)
       
pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_user_id_list_folders_COMPLETE.pickle'
with open(pickle_name, 'rb') as handle:
        dict_user_list_folders = pickle.load(handle)
print "read:",pickle_name,  len(dict_user_list_folders)
    
 




(6527213, 7)
after removing rows without dates:
  num. users: 175587 (5599544, 7)
438284
(438284, 24)
read: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_list_users_COMPLETE.pickle 519045
read: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_user_id_list_folders_COMPLETE.pickle 438284


In [ ]:
#df_users.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'}, inplace=True)


In [ ]:

# dict_user_id_user_attr   
# # 629145600.0: {'Country': 'United States',
# #   'University_name': 'cornell university',
# #   'career_stage': nan,
# #   'category_total_cit': nan,
# #   'category_total_last_auth': nan,
# #   'category_total_publ': nan,
# #   'eff_num_folders': 0,
# #   'email_domain': 'cornell.edu',
# #   'field': nan,
# #   'folders_above_avg': {1369112819.0: 0},
# #   'geoloc': '(42.4534492, -76.4735027) 14850',
# #   'gini_act_across_folders': nan,
# #   'group_num_citations': nan,
# #   'group_num_papers_last': nan,
# #   'group_total_publ': nan,
# #   'national_ranking': 8.0,
# #   'number_folders': 1,
# #   'user_id': 629145600.0,
# #   'user_tot_act': 0.0,
# #   'user_tot_num_adds': 0.0,
# #   'user_tot_num_deletes': 0.0,
# #   'user_tot_num_edits': 0.0,
# #   'user_univ_ranking': 10.0,
# #   'world_ranking': 10.0},



In [ ]:
# #### time window for temporal networks
# string_start_date='2014-05-01'
# start_date=pd.Timestamp(string_start_date)

# #string_end_date='2014-11-01'
# end_date=start_date + pd.Timedelta('180 days')




# list_tupla_dates=[]
# for i in range(6):
#     print start_date, end_date    
#     tupla_dates=[start_date,end_date]
#     list_tupla_dates.append(tupla_dates)

    
#     start_date += pd.Timedelta('180 days')
#     end_date += pd.Timedelta('180 days')

# # 2014-05-01_2014-10-28



### for plotting time series directly from data
#df.plot(x='folder_creation_date',y='folder_lifespan',ls='',marker='.') 


In [3]:

def remove_nans(lista):
   # print "\n",lista
    aux_lista=[]
    for i in range(len(lista)):    
       # print lista[i]
        try:
            if np.isnan(lista[i]) == True:  # (when nan):   # OJO!!! NO SIRVE SI HAGO: if lista[I]== np.nan   :(   !!!
                pass#print "nan found"
            else: # (when not-nan):           
                aux_lista.append(lista[i])
            #print "no problem"
        except : ## whenever it is a STR            
            pass            
            
            
    #print aux_lista   
    return aux_lista


############################3



def remove_nans_for_strings(lista):
   # print "\n",lista
    aux_lista=[]
    for i in range(len(lista)):    
       # print lista[i]
        try:
            if np.isnan(lista[i]) == True:  # (when nan):   # OJO!!! NO SIRVE SI HAGO: if lista[I]== np.nan   :(   !!!
                pass#print "nan found"
            else: # (when not-nan):           
                aux_lista.append(lista[i])
            #print "no problem"
        except : ## whenever it is a STR            
             aux_lista.append(lista[i])#pass             
            
            
    #print aux_lista   
    return aux_lista




###############

def remove_nans_replace_by_zeros(lista):
   # print "\n",lista
    aux_lista=[]
    for i in range(len(lista)):    
       # print lista[i]
        try:
            if np.isnan(lista[i]) == True:  # (when nan):   # OJO!!! NO SIRVE SI HAGO: if lista[I]== np.nan   :(   !!!
                aux_lista.append(0.)        #pass#print "nan found"
            else: # (when not-nan):           
                aux_lista.append(float(lista[i]))
            #print "no problem"
        except : ## whenever it is a STR            
            pass             
            
            
    #print aux_lista   
    return aux_lista




###### get dict with  folder_id lifespan:
def remove_nans_replace_intervals_with_starting_value(lista):
    #print "\n",lista
    aux_lista=[]
    for i in range(len(lista)):    
       # print lista[i]
        try:
            if np.isnan(lista[i]) == True:  # (when/ nan):   # OJO!!! NO SIRVE SI HAGO: if lista[I]== np.nan   :(   !!!
                pass#print "nan found"
        except TypeError: # (when not-nan):
            #print lista[i]   
            try:  
                aux_lista.append(float(lista[i]))
                #print "no problem"
            except ValueError:  # sometimes, the ranking is 75-82
                #print "issue found"
                value=lista[i]
#                 v1=float(value.split("-")[0])
#                 v2=float(value.split("-")[1])
                
                new_value=float(value.split("-")[0])#np.median([v1,v2])  
                aux_lista.append(new_value)
            
    #print aux_lista   
    return aux_lista


#############



def gini(old_list_of_values):
    
    list_of_values=[]
    for item in old_list_of_values:
        try:
            int(item) # if it is a NAN, it with fail
            list_of_values.append(item)
        except:
            pass
    
   # print list_of_values
    sorted_list = sorted(list_of_values)
    height, area = 0, 0
    for value in sorted_list:
        height += value
        area += height - value / 2.
    fair_area = height * len(list_of_values) / 2.
    try:
        return (fair_area - area) / fair_area
    except ZeroDivisionError:
#         print "problems with:",list_of_values   # if lista=[0,0,0]  or [0]
#         raw_input()
        return np.nan


##############


def  effective_num(lista_values):  # i can use it for the effective number of members in a folder, by activity, or the effective number of folders for a user, also by activity
    
     # first, i need to remove the zeros, because i cant do the log(0), but i can remove them, they dont count as effective members
    
    
    cont_num_nonzero_items=0
    H=0.
    tot_sum=float(sum(lista_values))
    for item in lista_values:
        if item >0:
            aux= item/tot_sum * np.log2( item/tot_sum)
            H += aux
            cont_num_nonzero_items +=1
            
    H = -1.0 * H
    
    eff_number = np.power(2.0, H)
    
    if cont_num_nonzero_items ==0:   # if the list of act is [0] then i want the eff. number to be 0, not 1
        eff_number =0
    
    return eff_number
        


In [4]:
     
        
dict_folder_dict_user_act_in_folder={}

for folder_id in tqdm_notebook(dict_folder_list_users):  

        list_users_in_folder=dict_folder_list_users[folder_id]
        
        for user_id in list_users_in_folder:
            
       
            df_select = df[(df['folder_id'] == folder_id )  & (df['user_id']==user_id )]
    
           
        
            num_adds=sum(remove_nans_replace_by_zeros(list(df_select.num_adds))) # for activity records, NAN = 0
            num_edits=sum(remove_nans_replace_by_zeros(list(df_select.num_edits)))
            num_deletes=sum(remove_nans_replace_by_zeros(list(df_select.num_dels)))
            user_act=num_adds+num_edits+num_deletes
      

        try: 
            dict_folder_dict_user_act_in_folder[folder_id]
        except KeyError:
            dict_folder_dict_user_act_in_folder[folder_id]={}            

        dict_folder_dict_user_act_in_folder[folder_id][user_id]=user_act

        

print "done with the dicts., size:",len(dict_folder_list_users),len(dict_folder_dict_user_act_in_folder)






path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/"
filename="dict_folder_dict_user_act_in_folder_PARTIAL.pickle"
with open(path+filename,'wb') as f:
    pickle.dump(dict_folder_dict_user_act_in_folder, f)
print "written:",path+filename




KeyboardInterrupt: 

In [ ]:

# ######## load dictionary of dict for users' act                            
# pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/new_dict_folder_dict_user_act_in_folder_more_attr_COMPLETE.pickle'
# with open(pickle_name, 'rb') as handle:
#     dict_folder_dict_user_act_in_folder_more_attr = pickle.load(handle)
# print len(dict_folder_dict_user_act_in_folder_more_attr.keys())




In [ ]:
# pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/new_dict_folder_dict_user_act_in_folder.pickle'
# with open(pickle_name, 'rb') as handle:
#     dict_folder_dict_user_act_in_folder = pickle.load(handle)
# print len(dict_folder_dict_user_act_in_folder.keys())

dict_folder_dict_user_act_in_folder=dict_folder_dict_user_act_in_folder_more_attr

In [ ]:
# dict_user_id_user_attr
# 1053958.0: {'Country': 'United States',
#   'University_name': 'university of washington',
#   'career_stage': nan,
#   'category_total_cit': nan,
#   'category_total_last_auth': nan,
#   'category_total_publ': nan,
#   'eff_num_folders': 1.5069930657725379,
#   'field': 'science',
#   'gini_act_across_folders': 0.8571428571428571,
#   'number_folders': 9,
#   'user_tot_act': 7.0,
#   'user_tot_num_adds': 4.0,
#   'user_tot_num_deletes': 3.0,
#   'user_tot_num_edits': 0.0,
#   'user_univ_ranking': 31.0},


In [ ]:
######## unweighted network:
list_missing_nodes=[]
G=nx.Graph()
for folder_id in tqdm_notebook(dict_folder_list_users):
        #dict_folder_list_users[folder]=list(set(dict_folder_list_users[folder]))  # remove possible duplicates       (there shouldnt be any)

        lista=dict_folder_list_users[folder_id]

        if len(lista)>1:

            lista_pares=itertools.combinations(lista, 2)        

            for item in lista_pares:
                e1=item[0]
                e2=item[1]
                G.add_edge(e1,e2)
                
                try: 
                    G.edge[e1][e2]["num_common_projects"] +=1
                except KeyError:
                    G.edge[e1][e2]["num_common_projects"] =1

    
        else:
            
            n=lista[0]
            G.add_node(n)
            try:  # because of the partial dict of user's attributes 
                G.node[n]["tot_act"]=dict_user_id_user_attr[n]['user_tot_act']
            except KeyError:
                list_missing_nodes.append(n)
                        

print "  N:", len(G.nodes()),"  L:", len(G.edges())       
GC = max(nx.connected_component_subgraphs(G), key=len)
print "\n  GC:     N:", len(GC.nodes()), "  L:", len(GC.edges())




path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/Networks/"
filename="network_all_new_PARTIAL.pickle"
with open(path+filename,'wb') as f:
    pickle.dump(G, f)
print "written:",path+filename




In [ ]:
lista=[]
for folder_id in dict_folder_list_users:
    for user in dict_folder_list_users[folder_id]:
        lista.append(user)
        
print len(set(lista)), len(dict_user_id_user_attr), len(list_missing_nodes)

In [ ]:
df_users

In [ ]:
print len(dict_folder_list_users), len(dict_folder_list_users), len(list_missing_nodes)

In [ ]:

print df_users.user_tot_act.mean(), df_users.user_tot_act.quantile(q=0.6)
act_threshold=1 #  28 is the 60% ,  56 is the 65%  ,   168 is the 70% percentile (50% percentile is 0 and 55% is also 0)

print "thershold for activity",act_threshold

list_missing_nodes=[]



####### weighted network:

G_weighted=nx.Graph()
for folder_id in tqdm_notebook(dict_folder_list_users):
       
                    
        lista_users=dict_folder_list_users[folder_id]
       
        weighted_list_users=[]
        for user_id in lista_users:
            
            try:
                user_act_in_folder=dict_folder_dict_user_act_in_folder[folder_id][user_id]
                if user_act_in_folder >= act_threshold:
                    weighted_list_users.append(user_id)
            except KeyError:
                list_missing_nodes.append(user_id)
                pass
                
      
        
        if len(weighted_list_users)>1:  # ignore nodes that havent done any work in a particular folder (they will only count as isolated)

            lista_pares=itertools.combinations(weighted_list_users, 2)        

            for item in lista_pares:
                e1=item[0]
                e2=item[1]
                G_weighted.add_edge(e1,e2)               
                
                try: 
                    G_weighted.edge[e1][e2]["num_common_projects"] +=1
                except KeyError:
                    G_weighted.edge[e1][e2]["num_common_projects"] =1
                    

                    
        
        for n in lista_users:
            G_weighted.add_node(n)
            try:  # because it the partial  dict for users' attr there are users missing
                G_weighted.node[n]["tot_act"]=dict_user_id_user_attr[n]['user_tot_act']
            except KeyError:
                pass
           

            
           
        

print "  N:", len(G_weighted.nodes()),"  L:", len(G_weighted.edges())       
GC = max(nx.connected_component_subgraphs(G_weighted), key=len)
print "\n  GC:     N:", len(GC.nodes()), "  L:", len(GC.edges())





path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/Networks/"
filename="network_all_weighted_by_act_thresh"+str(act_threshold)+"_PARTIAL.pickle"
with open(path+filename,'wb') as f:
    pickle.dump(G_weighted, f)
print "written:",path+filename


In [ ]:
len(list_missing_nodes)

In [ ]:








#### the master dict is larger than G.nodes(), and the set_node_attribute does not work in that case, so i get a selection of the dict only with the common nodes:
#######################

# new_dict_user_id = {k: dict_user_id_domain[k] for k in set(G.nodes()) & set(dict_user_id_domain.keys())}
# print len(G.nodes()), len(new_dict_user_id)
# # i add the email as node attributes    
# nx.set_node_attributes(G, 'email', new_dict_user_id)
# ############


